In [2]:
from pynq.overlays.base import BaseOverlay
from pynq_peripherals import ArduinoSEEEDGroveAdapter
base = BaseOverlay('base.bit')
adapter = ArduinoSEEEDGroveAdapter(base.ARDUINO, D7='grove_led_stick')
led_stick = adapter.D7
from qnn import *
from .dorefanet import Dorefanet
from .tinieryolo import TinierYolo
import sys
import os, platform
import json
import numpy as np
import cv2
import ctypes
from matplotlib import pyplot as plt
from PIL import Image
from datetime import datetime
from qnn import tinieryolo
from qnn import utils 
sys.path.append("/opt/darknet/python/")
from darknet import *
%matplotlib inline

import IPython
classifier = TinierYolo()
classifier.init_accelerator()
net = classifier.load_network(json_layer="/usr/local/lib/python3.6/dist-packages/qnn/params/tinier-yolo-layers.json")

conv0_weights = np.load('/usr/local/lib/python3.6/dist-packages/qnn/params/tinier-yolo-conv0-W.npy', encoding="latin1", allow_pickle=True)
conv0_weights_correct = np.transpose(conv0_weights, axes=(3, 2, 1, 0))
conv8_weights = np.load('/usr/local/lib/python3.6/dist-packages/qnn/params/tinier-yolo-conv8-W.npy', encoding="latin1", allow_pickle=True)
conv8_weights_correct = np.transpose(conv8_weights, axes=(3, 2, 1, 0))
conv0_bias = np.load('/usr/local/lib/python3.6/dist-packages/qnn/params/tinier-yolo-conv0-bias.npy', encoding="latin1", allow_pickle=True)
conv0_bias_broadcast = np.broadcast_to(conv0_bias[:,np.newaxis], (net['conv1']['input'][0],net['conv1']['input'][1]*net['conv1']['input'][1]))
conv8_bias = np.load('/usr/local/lib/python3.6/dist-packages/qnn/params/tinier-yolo-conv8-bias.npy', encoding="latin1", allow_pickle=True)
conv8_bias_broadcast = np.broadcast_to(conv8_bias[:,np.newaxis], (125,13*13))

file_name_cfg = c_char_p("/usr/local/lib/python3.6/dist-packages/qnn/params/tinier-yolo-bwn-3bit-relu-nomaxpool.cfg".encode())

net_darknet = lib.parse_network_cfg(file_name_cfg)
img_folder = './yoloimages/'
img_file = os.path.join(img_folder, random.choice(os.listdir(img_folder)))
file_name = c_char_p(img_file.encode())

img = load_image(file_name,0,0)
img_letterbox = letterbox_image(img,416,416)
img_copy = np.copy(np.ctypeslib.as_array(img_letterbox.data, (3,416,416)))
img_copy = np.swapaxes(img_copy, 0,2)
free_image(img)
free_image(img_letterbox)
img_copy = img_copy[np.newaxis, :, :, :]
    
conv0_ouput = utils.conv_layer(img_copy,conv0_weights_correct,b=conv0_bias_broadcast,stride=2,padding=1)
conv0_output_quant = conv0_ouput.clip(0.0,4.0)
conv0_output_quant = utils.quantize(conv0_output_quant/4,3)
out_dim = net['conv7']['output'][1]
out_ch = net['conv7']['output'][0]

conv_output = classifier.get_accel_buffer(out_ch, out_dim)
conv_input = classifier.prepare_buffer(conv0_output_quant*7);
classifier.inference(conv_input, conv_output)
conv7_out = classifier.postprocess_buffer(conv_output)
conv7_out_reshaped = conv7_out.reshape(out_dim,out_dim,out_ch)
conv7_out_swapped = np.swapaxes(conv7_out_reshaped, 0, 1) # exp 1
conv7_out_swapped = conv7_out_swapped[np.newaxis, :, :, :] 
conv8_output = utils.conv_layer(conv7_out_swapped,conv8_weights_correct,b=conv8_bias_broadcast,stride=1)  
conv8_out = conv8_output.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
lib.forward_region_layer_pointer_nolayer(net_darknet,conv8_out)
tresh = c_float(0.3)
tresh_hier = c_float(0.5)
file_name_out = c_char_p("/home/xilinx/jupyter_notebooks/qnn/detection".encode())
file_name_probs = c_char_p("/home/xilinx/jupyter_notebooks/qnn/probabilities.txt".encode())
file_names_voc = c_char_p("/opt/darknet/data/voc.names".encode())
darknet_path = c_char_p("/opt/darknet/".encode())
lib.draw_detection_python(net_darknet, file_name, tresh, tresh_hier,file_names_voc, darknet_path, file_name_out, file_name_probs);
file_content = open(file_name_probs.value,"r").read().splitlines()
detections = []
for line in file_content[0:]:
    name, probability = line.split(": ")
    detections.append((probability, name))
for det in sorted(detections, key=lambda tup: tup[0], reverse=True):
    print("class: {}\tprobability: {}".format(det[1], det[0]))
res = Image.open(file_name_out.value.decode() + ".png")
res
probability = int(file_content[8:10])
if probability == "aeroplane":
    led_stick.clear()
    led_stick.set_pixel(0, 0xFF0000)
    led_stick.set_pixel(1, 0xFF0000)
    led_stick.set_pixel(2, 0xFF0000)
    led_stick.set_pixel(3, 0xFF0000)
    led_stick.set_pixel(4, 0xFF0000)
    led_stick.set_pixel(5, 0xFF0000)
    led_stick.set_pixel(6, 0xFF0000)
    led_stick.set_pixel(7, 0xFF0000)
    led_stick.set_pixel(8, 0xFF0000)
    led_stick.set_pixel(9, 0xFF0000)
elif probability == "bicycle":
    led_stick.clear()
    led_stick.set_pixel(0, 0xFF0000)
    led_stick.set_pixel(1, 0xFF0000)
    led_stick.set_pixel(2, 0xFF0000)
    led_stick.set_pixel(3, 0xFF0000)
    led_stick.set_pixel(4, 0xFF0000)
    led_stick.set_pixel(5, 0xFF0000)
    led_stick.set_pixel(6, 0xFF0000)
    led_stick.set_pixel(7, 0xFF0000)
    led_stick.set_pixel(8, 0xFF0000)
    led_stick.set_pixel(9, 0xFF0000)
elif probability == "boat":
    led_stick.clear()
    led_stick.set_pixel(0, 0xFF0000)
    led_stick.set_pixel(1, 0xFF0000)
    led_stick.set_pixel(2, 0xFF0000)
    led_stick.set_pixel(3, 0xFF0000)
    led_stick.set_pixel(4, 0xFF0000)
    led_stick.set_pixel(5, 0xFF0000)
    led_stick.set_pixel(6, 0xFF0000)
    led_stick.set_pixel(7, 0xFF0000)
    led_stick.set_pixel(8, 0xFF0000)
    led_stick.set_pixel(9, 0xFF0000)
elif probability == "bus":
    led_stick.clear()
    led_stick.set_pixel(0, 0xFF0000)
    led_stick.set_pixel(1, 0xFF0000)
    led_stick.set_pixel(2, 0xFF0000)
    led_stick.set_pixel(3, 0xFF0000)
    led_stick.set_pixel(4, 0xFF0000)
    led_stick.set_pixel(5, 0xFF0000)
    led_stick.set_pixel(6, 0xFF0000)
    led_stick.set_pixel(7, 0xFF0000)
    led_stick.set_pixel(8, 0xFF0000)
    led_stick.set_pixel(9, 0xFF0000)
elif probability == "car":
    led_stick.clear()
    led_stick.set_pixel(0, 0xFF0000)
    led_stick.set_pixel(1, 0xFF0000)
    led_stick.set_pixel(2, 0xFF0000)
    led_stick.set_pixel(3, 0xFF0000)
    led_stick.set_pixel(4, 0xFF0000)
    led_stick.set_pixel(5, 0xFF0000)
    led_stick.set_pixel(6, 0xFF0000)
    led_stick.set_pixel(7, 0xFF0000)
    led_stick.set_pixel(8, 0xFF0000)
    led_stick.set_pixel(9, 0xFF0000)
elif probability == "motorbike":
    led_stick.clear()
    led_stick.set_pixel(0, 0xFF0000)
    led_stick.set_pixel(1, 0xFF0000)
    led_stick.set_pixel(2, 0xFF0000)
    led_stick.set_pixel(3, 0xFF0000)
    led_stick.set_pixel(4, 0xFF0000)
    led_stick.set_pixel(5, 0xFF0000)
    led_stick.set_pixel(6, 0xFF0000)
    led_stick.set_pixel(7, 0xFF0000)
    led_stick.set_pixel(8, 0xFF0000)
    led_stick.set_pixel(9, 0xFF0000)
elif probability == "train":
    led_stick.clear()
    led_stick.set_pixel(0, 0xFF0000)
    led_stick.set_pixel(1, 0xFF0000)
    led_stick.set_pixel(2, 0xFF0000)
    led_stick.set_pixel(3, 0xFF0000)
    led_stick.set_pixel(4, 0xFF0000)
    led_stick.set_pixel(5, 0xFF0000)
    led_stick.set_pixel(6, 0xFF0000)
    led_stick.set_pixel(7, 0xFF0000)
    led_stick.set_pixel(8, 0xFF0000)
    led_stick.set_pixel(9, 0xFF0000)
else:
    led_stick.clear()
    led_stick.set_pixel(0, 0x00FF00)
    led_stick.set_pixel(1, 0x00FF00)
    led_stick.set_pixel(2, 0x00FF00)
    led_stick.set_pixel(3, 0x00FF00)
    led_stick.set_pixel(4, 0x00FF00)
    led_stick.set_pixel(5, 0x00FF00)
    led_stick.set_pixel(6, 0x00FF00)
    led_stick.set_pixel(7, 0x00FF00)
    led_stick.set_pixel(8, 0x00FF00)
    led_stick.set_pixel(9, 0x00FF00)
    led_stick.show()
    

ImportError: cannot import name 'Xlnk' from 'pynq' (/usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/__init__.py)